In [1]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_4.pkl'
orientation_R = '../calibration/orientation_2.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)


In [3]:
import numpy as np
actual_point = [0, 0, -150, 1]
zero_point_L = (592, 530)
zero_point_R = (558, 698)

point = (zero_point_L, zero_point_R)

projection = triangulator.projection_matrices[0]
cam_matr = triangulator.camera_matrices[0]

# get position with triangulators constructed projection matrix
position_proj = (projection @ actual_point)
position_proj = position_proj / position_proj[-1]
project_right = triangulator.projection_matrices[1] @ actual_point
project_right = project_right / project_right[-1]

print('position from projection matrix: ', position_proj)
print('position from right proj matrix: ', project_right)

print('triangulated zero point: ', triangulator.triangulate(point))

position from projection matrix:  [581.56786281 545.1872889    1.        ]
position from right proj matrix:  [555.18191316 713.1679173    1.        ]
homo:  [ 1.13776375e-03 -4.13356253e-01 -9.71079422e-04  1.00000000e+00]
triangulated zero point:  [ 0.00113776 -0.41335625 -0.00097108]


In [4]:
A = triangulator.get_DLT_matrix(point)
print('A: \n', A)
U, S, Vt = np.linalg.svd(A, full_matrices=False)
print('s: \n', S)
print('Vt: \n', Vt)
vn = Vt[3, :]
print('vn: \n', vn.reshape((4, 1)))
print('vn normalized: \n', (vn / vn[-1]).reshape((4, 1)))
print('all close: ', np.allclose(U @ np.diag(S) @ Vt, A))
print('w2: ', vn.T @ A.T @ A @ vn)

# reproject
reproj = projection @ vn / vn[-1]
reproj = reproj / reproj[-1]
print('triangulated back: \n', reproj)

A: 
 [[ 5.57402195e+05 -6.45001853e+05  1.21170403e+05  2.93953242e+08]
 [ 1.20121896e+03 -6.64431871e+02  8.13290341e+01  2.84906729e+03]
 [ 6.03964397e+05  8.23143856e+05  4.84845839e+05  3.91369619e+08]
 [ 4.55432799e+02  1.26664041e+03  3.27338913e+02  4.67793424e+04]]
s: 
 [4.89468735e+08 1.03193528e+06 1.25413789e+03 3.75245676e+01]
Vt: 
 [[-1.67052703e-03 -5.53277566e-04 -9.40701566e-04 -9.99998009e-01]
 [-8.04072795e-02  9.78818412e-01  1.88278637e-01 -5.84350906e-04]
 [-9.83434057e-01 -4.71186718e-02 -1.75025498e-01  1.83357329e-03]
 [ 1.62447945e-01  1.99233636e-01 -9.66393473e-01  5.27483719e-04]]
vn: 
 [[ 1.62447945e-01]
 [ 1.99233636e-01]
 [-9.66393473e-01]
 [ 5.27483719e-04]]
vn normalized: 
 [[ 3.07967695e+02]
 [ 3.77705754e+02]
 [-1.83208209e+03]
 [ 1.00000000e+00]]
all close:  True
w2:  1408.0931732815748
triangulated back: 
 [-9.29925644e+01 -4.92879446e+04  1.00000000e+00]


In [7]:
# Temuges DLT
point1 = zero_point_L
point2 = zero_point_R
P1 = triangulator.projection_matrices[0]
P2 = triangulator.projection_matrices[1]
A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
A = np.array(A).reshape((4, 4))
B = A.transpose() @ A
U, S, Vh = np.linalg.svd(B, full_matrices=False)
print('triangulated point: ')
print(Vh[3, 0:3] / Vh[3, 3])

triangulated point: 
[   7.01551484    0.50999278 -159.40488786]


In [20]:
p1, p2, p3 = tuple(triangulator.projection_matrices[0])
# print(p1)
# print(p2)
# print(p3)
print(triangulator.projection_matrices[0])

[[ 1.08309071e+03 -1.18985701e+03  2.28652066e+02  5.40099812e+05]
 [-2.12327677e+01 -1.33766810e+02  1.41676714e+03  7.01315580e+05]
 [-2.27765129e-01 -9.39106497e-01  2.57297559e-01  9.42790307e+02]]


In [ ]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

In [1]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

NameError: name 'caps' is not defined

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()